In [1]:
from keras.models import model_from_json
import numpy as np
from PIL import Image
import keyboard
import time
from mss import mss

In [2]:
pos = {"top": 460, "left": 700, "width": 300, "height": 125}

sct = mss()

In [3]:
width = 125
height = 50

## 1-Read Model

In [4]:
model = model_from_json(open("model.json","r").read())
model.load_weights("model.h5")

In [5]:
# down = 0, right = 1, up = 2 => right önemsiz hiçbir şey yapma demek
labels = ["Down", "Right", "Up"] # klavyeye basmadan klavyeye basmış gibi komut yollayacağız.

## 2-Test Data

In [6]:
framerate_time = time.time()
counter = 0
i = 0
delay = 0.4 # bir tuşa basında 0.4 s bekliyoruz.
key_down_pressed = False

while True:
    
    img = sct.grab(pos) # catch pixel
    im = Image.frombytes("RGB", img.size, img.rgb) # convert RGB
    im2 = np.array(im.convert("L").resize((width, height))) # resize image
    im2 = im2 / 255 # normalize
    
    X = np.array([im2])
    X = X.reshape(X.shape[0], width, height, 1) # her döngüde bir image okunur
    r = model.predict(X) # [0.1, 0.5, 0.4] = toplamı 1
    
    result = np.argmax(r) # Yukarıdaki örnekte en büyüğü 0.5'tu. Indexi 1'dir.
    
    if result == 0: # down = 0
        
        keyboard.press(keyboard.KEY_DOWN)
        key_down_pressed = True
        
    elif result == 2:    # up = 2
        
        if key_down_pressed:
            keyboard.release(keyboard.KEY_DOWN)
        time.sleep(delay)
        keyboard.press(keyboard.KEY_UP)
        
        if i < 1500:
            time.sleep(0.3)
        elif 1500 < i and i < 5000:  # 1500. frameden sonra oyun hızlanıyor. zıplama süresini azaltmak lazım. 
            time.sleep(0.2) # Yere daha çabuk iner.
        else: # 5000'den fazla ise oyun daha da hızlanır. 
            time.sleep(0.17) # Yere daha da çabuk inmesi için azaltırız.
            
        # yukarı bastıktan sonra aşağı basarak initial postiona dönmesi sağlanır.
        keyboard.press(keyboard.KEY_DOWN)
        keyboard.release(keyboard.KEY_DOWN) #aşağı bastıktan sonra ilaveten release yapmak gerek /basılı kalır yoksa.
    
    counter += 1
    
    if (time.time() - framerate_time) > 1:
        
        counter = 0
        framerate_time = time.time()
        if i <= 1500: # 1500. frame
            delay -= 0.003
        else:
            delay -= 0.005
        if delay < 0:
            delay = 0
            
        print("---------------------")
        print("Down: {} \nRight:{} \nUp: {} \n".format(r[0][0],r[0][1],r[0][2]))
        i += 1

---------------------
Down: 0.019012287259101868 
Right:0.12628251314163208 
Up: 0.8547052145004272 

---------------------
Down: 0.019012287259101868 
Right:0.12628251314163208 
Up: 0.8547052145004272 

---------------------
Down: 0.019012287259101868 
Right:0.12628251314163208 
Up: 0.8547052145004272 

---------------------
Down: 0.1357530802488327 
Right:0.747625470161438 
Up: 0.1166214570403099 

---------------------
Down: 0.13197144865989685 
Right:0.7503579258918762 
Up: 0.11767057329416275 

---------------------
Down: 0.16108964383602142 
Right:0.7570393085479736 
Up: 0.08187098056077957 

---------------------
Down: 0.1285669058561325 
Right:0.7510509490966797 
Up: 0.1203821673989296 

---------------------
Down: 1.522068146186939e-06 
Right:0.0001523734099464491 
Up: 0.9998461008071899 

---------------------
Down: 0.6701844334602356 
Right:0.3287932276725769 
Up: 0.0010223761200904846 

---------------------
Down: 0.6701844334602356 
Right:0.3287932276725769 
Up: 0.00102237